# Evaluation QAOA

In [ ]:
from mqt.problemsolver.partialcompiler import Partial_QAOA

In [ ]:
from mqt.problemsolver.partialcompiler import evaluator
evaluator.evaluate_QAOA(num_qubits=7, repetitions=5)

In [ ]:
p = Partial_QAOA.Partial_QAOA(15,2)
(
qc_prep_nat_gates,
qcs_problem_nat_gates,
qcs_mixer_nat_gates, 
) = p.get_partially_compiled_circuit_without_online_edges()

mapping = p.mapping
print("old mapping:", mapping)

In [ ]:
qcs_problem_nat_gates[0].draw(fold=-1, output="mpl")

In [ ]:
tmp = qcs_problem_nat_gates[0]._layout.final_layout
if tmp is not None:
    lut = tmp.get_physical_bits()
    for elem in mapping:
        if elem != lut[elem].index:
            print(elem, "->", lut[elem].index)

In [ ]:
from typing import Literal, Union, overload

import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.circuit import Parameter
from qiskit.providers.fake_provider import FakeManila, FakeMontreal, FakeWashington

In [ ]:
offline_mapped_qc = transpile(
    qc,
    coupling_map=FakeMontreal().configuration().coupling_map,
    basis_gates=FakeMontreal().configuration().basis_gates,
    optimization_level=3,
)

In [ ]:
offline_mapped_qc._layout.final_layout

In [ ]:
qcs_problem_nat_gates[1]._layout

In [ ]:
qcs_problem_nat_gates[1].draw()

In [ ]:
i = qcs_problem_nat_gates[1]._layout.initial_layout
i

In [ ]:
i.get_virtual_bits()

In [ ]:
f.get_virtual_bits()

In [ ]:
for j, key in enumerate(f):
    print(j)
    if j == qcs_problem_nat_gates[1].num_qubits-1:
        break

In [ ]:
qc_prep_nat_gates.draw()

In [ ]:
from qiskit import QuantumCircuit, transpile

In [ ]:
qc = QuantumCircuit(4)
qc.x([1,2,0])
qc.cx(0,1)
#qc.cx(1,2)
#qc.cx(0,2)
#qc.cx(0,3)
qc.draw()

In [ ]:
from qiskit.providers.fake_provider import FakeManila

In [ ]:
qc_transpiled = transpile(qc, backend=FakeManila(), optimization_level=0)
qc_transpiled.draw()

In [ ]:
qc_transpiled._layout

# Backup


In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from qiskit_optimization.applications import Maxcut

In [ ]:
n=4 # Number of nodes in graph
G=nx.Graph()
G.add_nodes_from(np.arange(0,n,1))
elist=[(0,1,1),(0,2,1.0),(2,3,1.0),(1,2,1.0)] # ,(0,2,1.0)
# tuple is (i,j,weight) where (i,j) is the edge
G.add_weighted_edges_from(elist)
colors = ['r' for node in G.nodes()]
pos = nx.spring_layout(G)

In [ ]:
def draw_graph(G, colors, pos):
    default_axes = plt.axes(frameon=True)
    nx.draw_networkx(G, node_color=colors, node_size=600, alpha=.8, ax=default_axes, pos=pos)
    edge_labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos=pos, edge_labels=edge_labels)

draw_graph(G, colors, pos)

In [ ]:
w = np.zeros([n,n])
for i in range(n):
    for j in range(n):
        temp = G.get_edge_data(i,j,default=0)
        if temp != 0:
            w[i,j] = temp['weight']
print(w)

In [ ]:
max_cut = Maxcut(w)
qp = max_cut.to_quadratic_program()
print(qp)

In [ ]:
from qiskit.algorithms.minimum_eigen_solvers import NumPyMinimumEigensolver
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.algorithms.minimum_eigensolvers import SamplingVQE
from qiskit.algorithms.optimizers import SPSA
from qiskit.circuit.library import TwoLocal
from qiskit.primitives import Sampler
from qiskit.algorithms.minimum_eigensolvers import QAOA
from qiskit import transpile

# Different Solver

In [ ]:
qubitOp, offset = qp.to_ising()
exact = MinimumEigenOptimizer(NumPyMinimumEigensolver())
result = exact.solve(qp)
result

In [ ]:
optimizer = SPSA(maxiter=300)
ry = TwoLocal(qubitOp.num_qubits, "ry", "cz", reps=1, entanglement="linear")
vqe = SamplingVQE(sampler=Sampler(), ansatz=ry, optimizer=optimizer)

# run SamplingVQE
result = vqe.compute_minimum_eigenvalue(qubitOp)
result

In [ ]:
qaoa = QAOA(optimizer=SPSA(), sampler=Sampler())
res = qaoa.compute_minimum_eigenvalue(qubitOp)
print(res)